<a href="https://colab.research.google.com/github/MarkDvoryanchikov/year_teachers_hours_01.06.20_to_01.06.21/blob/main/Moodle_year_logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
pd.set_option('display.max_rows', 10)

In [3]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/nir_files/teachers_year.csv', sep=';')


# logs data import from lms moodle to csv
"""
SELECT id, action, userid, timecreated  FROM moodleday.mdl_logstore_standard_log 
WHERE userid IN ( 
	SELECT userid FROM moodleday.mdl_cohort_members 
    WHERE cohortid IN ( 
		SELECT id FROM moodleday.mdl_cohort WHERE name like 'Преподаватели' 
	)
    AND userid NOT IN ( -- нет нагрузки ВО
		2112, 2115, 2143, 2203, 2321, 2363, 2405, 2421, 3827, 3987,
		4082, 4709, 4718, 4775, 4792, 9644, 9672,11718,11887,12418,
		12776, 14083, 14895, 14896, 22, 517, 520, 536, 539, 574, 
		576, 596, 970, 972, 981, 14903, 14890, 14233, 14140, 12421,
		12422, 12423, 12424, 12426, 12427, 12428, 12429, 12702, 11912, 11682,
		9699, 9692, 9674, 8831 
	)
);
"""
data

,id,action,userid,timecreated
0,101420696,sent,11,1593051606
1,101421378,sent,11,1593051909
2,101421380,sent,11,1593051909
3,101422559,sent,11,1593052806
4,101422560,sent,11,1593052806
...,...,...,...,...
3253270,142328588,graded,14897,1620202481
3253271,146098318,graded,14897,1622010482
3253272,146106123,graded,14897,1622011735
3253273,146106504,graded,14897,1622011801


In [4]:
data = data[data['userid'] != 12782] # нет нагрузки ВО

In [5]:
TIME_OUT = 60*60*2 # 2 часа - время принудительного выхода из системы  
TIME_OUT # Если между событиями время больше TIME_OUT значит пользователь не активен 

COUNT_TEACHERS = 311 # Количество преподавателей ВО в LMS Moodle - 100%:
"""
SELECT count(*) FROM ( 
	SELECT userid FROM moodleday.mdl_cohort_members 
    WHERE cohortid IN ( 
		SELECT id FROM moodleday.mdl_cohort WHERE name like 'Преподаватели' 
	)
    AND userid NOT IN ( -- нет нагрузки ВО
		2112, 2115, 2143, 2203, 2321, 2363, 2405, 2421, 3827, 3987,
		4082, 4709, 4718, 4775, 4792, 9644, 9672,11718,11887,12418,
		12776, 14083, 14895, 14896, 22, 517, 520, 536, 539, 574, 
		576, 596, 970, 972, 981, 14903, 14890, 14233, 14140, 12421,
		12422, 12423, 12424, 12426, 12427, 12428, 12429, 12702, 11912, 11682,
		9699, 9692, 9674, 8831 
	)
) t;

"""

data.dtypes

id              int64
action         object
userid          int64
timecreated     int64
dtype: object

In [6]:
data = data.sort_values(['userid','timecreated']) # Упорядочиваем набор данных, что необходимо после склейки таблиц логов
data = data.set_index(['userid','id']) # Группировка по преподавателям ВО


In [7]:
data['Interval'] = data['timecreated'].diff() # Вычисляем расстояния между событиями
data = data.fillna(0) # Заполняем нулём первую строку для первого преподавателя ( убираем Null )
data.loc[data['Interval'] <0, 'Interval'] = 0 # Заполняем нулём первую строку для остальных преподавателей 
                                              # (убираем отрицательные значения в результате расчёта расстояний)
data

action  timecreated  Interval
userid id                                        
11     101108156  loggedin   1592806937       0.0
       101108164    viewed   1592806938       1.0
       101117733    viewed   1592809542    2604.0
       101118466    viewed   1592809773     231.0
       101139401  loggedin   1592816059    6286.0
...                    ...          ...       ...
14897  146106876    viewed   1622011852       1.0
       146107055      sent   1622011881      29.0
       146107057      sent   1622011881       0.0
       146107059      sent   1622011881       0.0
       146590193    viewed   1622160483  148602.0

[3253207 rows x 3 columns]

In [8]:
data_activities = data[data['Interval'] < TIME_OUT] # иcключаем периоды неактивности

In [9]:
data_activities['Interval'].sum()/60/60

51327.82833333334

In [10]:
data_activities['Interval'].sum()/60/60/COUNT_TEACHERS # часов на человека

165.04124866023582

In [11]:
# с учётом средней доли ставки 
AVERAGE_RATE = 0.895
data_activities['Interval'].sum()/60/60/COUNT_TEACHERS/AVERAGE_RATE

184.403629787973